In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
import pandas as pd
import time
import wandb
tqdm.pandas()
from datasets import load_dataset

# 使用lora进行微调
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model, prepare_model_for_int8_training

# trl-transformer reinforcement learning
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/intern/anaconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/intern/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/intern/anaconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/intern/anaconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/intern/anaconda3/lib/libcudart.so.11.0'), PosixPath('/home/intern/anaconda3/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [2]:
# configuration
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb", # Huggingface中利用imdb数据集对gpt2微调的模型 相当于SFT步骤
    learning_rate=1.4e-5, 
    steps=20000,
    batch_size=256,
    forward_batch_size=16,
    ppo_epochs=4,
    init_kl_coef=0.2,
    target=6, # Target KL value for adaptive KL control
    log_with="wandb", # 使用wandb监视训练过程，也可以使用tensorboard
)
#    log_with="tensorboard", # 使用wandb监视训练过程，也可以使用tensorboard
#    accelerator_kwargs={"logging_dir": "./tb_logger"} # 使用tensorboard加上这一行

wandb.init(name='run-imdb', project='gpt2-test', config=config, )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe_device = 0 if torch.cuda.is_available() else -1

/home/intern/anaconda3/lib/python3.10/site-packages/trl/trainer/ppo_config.py:112: UserWarning: Note that using `forward_batch_size` is deprecated, use `mini_batch_size` instead. By setting it you overwrite `mini_batch_size` which affects both the batch size during forward passes and also the mini batch size for PPO optimization.
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gaowenxuan101. Use `wandb login --relogin` to force relogin


In [3]:
# step 1: 加载模型

pretrained_model = AutoModelForCausalLM.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token # 设置pad_token和eos_token相同

# 设置目标模块名称
target_modules = None
target_modules = ["c_attn"]  # workaround to use 8bit training on this model

# 设置lora配置参数
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,  # handled automatically by peft
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
# step 2: 设置8bit训练
pretrained_model = prepare_model_for_int8_training(pretrained_model)

# step 3: 设置lora模型。做instruction learning，到这里就好了。如果要做RLHF，还要做第四步。
pretrained_model = get_peft_model(pretrained_model, lora_config)

# step 4: 将lora模型加载入trl模型，加上value head。
model = AutoModelForCausalLMWithValueHead.from_pretrained(pretrained_model)

# 做必要的设置，梯度检查。
model.gradient_checkpointing_disable = model.pretrained_model.gradient_checkpointing_disable
model.gradient_checkpointing_enable = model.pretrained_model.gradient_checkpointing_enable

/home/intern/anaconda3/lib/python3.10/site-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [4]:
# 加载了两次模型：第一次加载的模型用来进行强化学习，调整参数；第二次加载的模型作为参考模型。
# 计算两个模型的KL散度，用来作为PPO训练的额外奖励信号，来保证我们的模型不会太偏离原始模型（即防止灾难性遗忘情况的发生）。
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

wandb.watch(model, log='all') #观察模型

model.to(device)
ref_model.to(device)

AutoModelForCausalLMWithValueHead(
  (pretrained_model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
  (

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
    
print_trainable_parameters(model)

trainable params: 590593 || all params: 125030401 || trainable%: 0.47235951838625234


In [6]:
# 加载IMDB数据集 
def build_dataset(tokenizer, dataset_name='imdb', input_min_text_length=2, input_max_text_length=8):
    """ 
    Args:
        dataset_name (`str`): 
            数据集名称
    
    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            返回dataloader
    """
    # 加载IMDB数据集，从huggingface的hub上下载数据，当然也可以下载其他数据
    ds = load_dataset(dataset_name, split='train') # 加载后是DataFrame格式
    ds = ds.rename_columns({'text': 'review'})
    ds = ds.filter(lambda x: len(x["review"])>200, batched=False) # filter指len(x["review"])>200都过滤掉

    # 对batch_size进行裁剪，缩小到2到8之间。（2和8是函数中的默认参数）
    # 在tokenize之前，随机截断输入数据作为待续写的prompt，即query的token长度控制在2到8之间
    input_size = LengthSampler(input_min_text_length, input_max_text_length)
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[:input_size()] # 后面设置batched=False,每次input_size都不同
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    # 将数值型变量设置为torch的tensor格式，并且输出所有的列数据，在RL截断需要使用！一定要注意设置output_all_columns=True
    ds.set_format(type='torch', columns=["input_ids", "label"], output_all_columns=True)
    return ds

dataset = build_dataset(tokenizer)
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])


Using the latest cached version of the module from /home/intern/.cache/huggingface/modules/datasets_modules/datasets/imdb/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0 (last modified on Tue Jun 20 11:59:49 2023) since it couldn't be found locally at imdb., or remotely on the Hugging Face Hub.
Found cached dataset imdb (/home/intern/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached processed dataset at /home/intern/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-cd96d6dda9f74d63.arrow
Loading cached processed dataset at /home/intern/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-f92685a40095315f.arrow


In [7]:
# load一个pipeline影评分类器
sent_kwargs = {
    "return_all_scores": True, # 文本生成的参数，这里设置为True，表示生成文本时返回得分
    "function_to_apply": "none", 
    "batch_size": config.forward_batch_size 
}

# 加载在IMDB数据集上微调过的BERT分类器来得到拼接后文本的得分


sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=pipe_device)

# eg:
text = 'this movie was really bad!!'
sentiment_pipe(text, **sent_kwargs)

/home/intern/anaconda3/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 2.3350486755371094},
  {'label': 'POSITIVE', 'score': -2.726576566696167}]]

In [24]:
pipe_outputs = sentiment_pipe(text, **sent_kwargs)
[torch.tensor(output[1]["score"]) for output in pipe_outputs]
#torch.tensor([output[1]["score"] for output in pipe_outputs])

/home/intern/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


tensor(-2.7266)

In [15]:
# 配置PPO强化学习训练对象
ppo_trainer = PPOTrainer(config, model, ref_model=ref_model, 
                         tokenizer=tokenizer, dataset=dataset, 
                         data_collator=collator)

# 根据query生成response，这里的配置使用top_p和随机采样来生成文本。
generation_kwargs = {
    "min_length":-1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}

if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu" # to avoid a `pipeline` bug

训练循环主要包含三个步骤：

- 根据query，基于GPT2生成response
- 拼接query和response，使用BERT来得到拼接后文本的得分
- 基于(query, response, reward)三元组，基于PPO算法来优化模型

In [26]:
import numpy as np
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)
#output_length_sampler() #4-16随机

for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    logs, timing = dict(), dict()
    t0 = time.time()

    query_tensors = batch['input_ids']
    
    model.gradient_checkpointing_disable()
    model.pretrained_model.config.use_cache = True
    
    #### Get response from gpt2
    t = time.time()
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch['response'] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    timing['time/get_response'] = time.time() - t

    #### Compute sentiment score
    t = time.time()
    texts = [q + r for q,r in zip(batch['query'], batch['response'])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs] 
    # print(rewards)
    #若一个prompt目前是negative,它的positive score是-0.5，那么加到奖励里面，相当于让它少学这个
    timing['time/get_sentiment_preds'] = time.time()-t
    
    model.gradient_checkpointing_enable()
    model.pretrained_model.config.use_cache = False
    
    #### Run PPO step 
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    timing['time/PPOoptimization'] = time.time()-t
    
    # ppo_trainer.log_stats(stats, batch, rewards)
    if not isinstance(rewards, torch.Tensor):
        rewards = torch.tensor(rewards).to(device)


    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(batch['query'], batch['response'], rewards.cpu().tolist())]
    logs.update({'game_log': wandb.Table(columns=['query', 'response', 'reward'], rows=table_rows)})
    logs.update(timing)
    logs.update(stats)
    logs["env/reward_mean"] = torch.mean(rewards).cpu().numpy().item()
    logs["env/reward_std"] = torch.std(rewards).cpu().numpy().item()
    logs["env/reward_dist"] = rewards.cpu().numpy()
    wandb.log(logs)


0it [00:00, ?it/s]

/home/intern/anaconda3/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/home/intern/anaconda3/lib/python3.10/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[tensor(-0.8935), tensor(0.0518), tensor(2.9085), tensor(2.7588), tensor(1.9201), tensor(-0.1189), tensor(-0.2987), tensor(-1.6148), tensor(0.1520), tensor(0.4257), tensor(1.7670), tensor(1.5775), tensor(1.7510), tensor(-2.8840), tensor(0.6436), tensor(2.5403), tensor(-0.1962), tensor(0.5489), tensor(-0.7597), tensor(0.1947), tensor(-0.9156), tensor(0.7241), tensor(1.8682), tensor(1.0769), tensor(-1.9187), tensor(-0.4945), tensor(-1.4293), tensor(2.4175), tensor(1.9236), tensor(-1.3548), tensor(-2.0223), tensor(1.0798), tensor(1.8283), tensor(1.3926), tensor(-1.0926), tensor(-2.0928), tensor(2.2884), tensor(2.5798), tensor(0.7998), tensor(1.0777), tensor(-0.0826), tensor(0.5947), tensor(2.7151), tensor(-0.4463), tensor(2.2430), tensor(-1.7067), tensor(1.2143), tensor(2.2621), tensor(0.6188), tensor(2.3022), tensor(-0.4652), tensor(2.3229), tensor(0.5158), tensor(-1.0989), tensor(-1.8274), tensor(0.9576), tensor(-1.2952), tensor(2.5903), tensor(-0.1840), tensor(1.6222), tensor(0.6544), 

1it [02:15, 135.19s/it]

[tensor(-0.2435), tensor(1.0555), tensor(0.9231), tensor(1.5857), tensor(1.2387), tensor(-2.1317), tensor(-0.5956), tensor(0.1178), tensor(-2.2543), tensor(0.6694), tensor(-0.4683), tensor(-2.1079), tensor(1.2001), tensor(0.0794), tensor(2.6437), tensor(1.5276), tensor(-1.2377), tensor(0.1957), tensor(-2.8269), tensor(-0.8206), tensor(-0.8448), tensor(0.0493), tensor(1.9695), tensor(-0.6049), tensor(-0.9513), tensor(1.0887), tensor(0.9312), tensor(2.6901), tensor(-2.3310), tensor(1.5136), tensor(-0.0299), tensor(-1.7451), tensor(0.4410), tensor(-1.8486), tensor(1.5220), tensor(1.5010), tensor(-0.3027), tensor(0.8738), tensor(2.3072), tensor(1.9156), tensor(-0.3063), tensor(-0.7687), tensor(-1.3102), tensor(1.0039), tensor(0.0438), tensor(-0.8101), tensor(2.3237), tensor(2.6204), tensor(2.6550), tensor(-0.8812), tensor(0.1363), tensor(1.1081), tensor(0.0830), tensor(-0.0125), tensor(-1.9874), tensor(2.7879), tensor(2.6820), tensor(2.6530), tensor(1.9648), tensor(0.7160), tensor(-0.9871)

...

71it [2:41:01, 145.28s/it]

[tensor(2.0098), tensor(2.7417), tensor(2.5339), tensor(2.5625), tensor(1.1562), tensor(2.6632), tensor(2.2337), tensor(-0.5266), tensor(2.8749), tensor(2.7127), tensor(2.7081), tensor(2.7736), tensor(2.4437), tensor(-0.3596), tensor(2.0883), tensor(2.4557), tensor(2.6320), tensor(2.6928), tensor(0.4603), tensor(-1.4238), tensor(2.8114), tensor(2.7056), tensor(2.8933), tensor(1.9449), tensor(1.3754), tensor(2.8271), tensor(2.1520), tensor(2.5348), tensor(2.7621), tensor(2.7451), tensor(2.8326), tensor(2.6701), tensor(2.6719), tensor(2.7861), tensor(1.5178), tensor(2.8739), tensor(2.4832), tensor(1.8801), tensor(0.6066), tensor(2.0928), tensor(1.7165), tensor(-2.2921), tensor(2.8321), tensor(-1.0157), tensor(2.8295), tensor(2.7115), tensor(-2.8862), tensor(2.8770), tensor(2.3184), tensor(2.8252), tensor(0.8411), tensor(0.4814), tensor(2.4015), tensor(2.5625), tensor(2.6886), tensor(1.6736), tensor(2.9068), tensor(2.6710), tensor(2.6223), tensor(2.4768), tensor(2.8868), tensor(2.6164), t

72it [2:43:00, 137.46s/it]

[tensor(-0.1584), tensor(2.8067), tensor(2.8177), tensor(2.7576), tensor(2.6558), tensor(1.8963), tensor(2.6550), tensor(2.4918), tensor(2.2393), tensor(2.7099), tensor(2.2789), tensor(2.6742), tensor(1.5950), tensor(-1.0738), tensor(2.8290), tensor(2.8611), tensor(-0.8457), tensor(-1.1805), tensor(2.4079), tensor(2.6098), tensor(0.7579), tensor(2.8198), tensor(-1.0484), tensor(0.5761), tensor(1.9413), tensor(2.7647), tensor(1.6279), tensor(2.0910), tensor(1.0541), tensor(2.0114), tensor(2.7490), tensor(2.7836), tensor(1.6775), tensor(2.4321), tensor(2.3385), tensor(1.9295), tensor(1.2601), tensor(1.5545), tensor(2.3841), tensor(2.2940), tensor(2.8947), tensor(2.7453), tensor(2.7763), tensor(2.7415), tensor(2.6808), tensor(2.8715), tensor(2.4950), tensor(2.7666), tensor(1.7497), tensor(2.6625), tensor(2.4017), tensor(2.3951), tensor(2.7671), tensor(0.7932), tensor(2.5800), tensor(-1.2889), tensor(2.2976), tensor(2.7865), tensor(2.6245), tensor(2.9365), tensor(2.6230), tensor(2.7003), t

73it [2:45:21, 138.40s/it]